In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from geopy.distance import geodesic

In [2]:
%matplotlib agg



In [3]:
m = folium.Map(location = [0,0], zoom_start=2)
m

In [4]:
def create_map(location, zoom_start=10):
  map_object = folium.Map(location=location, zoom_start=zoom_start)
  return map_object

bulgaria_map = create_map([42.7339, 25.4858], zoom_start=7)
bulgaria_map

In [5]:
varna_map=create_map([43.2141, 27.9147], zoom_start=12)

In [6]:
varna_map

In [7]:
varna_map.save("varna.html")

In [8]:
varna_map.save("varna.jpg")

In [9]:
from ipyleaflet import Map

global_map = Map(center=(0,0), zoom=2)
global_map

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

In [10]:
from ipyleaflet import Map, TileLayer

m = Map(center=(0,0), zoom=2)
topo_layer = TileLayer(url='https://{s}.tile.opentopomap.org/{z}/{x}/{y}.png',
                       attribution = 'Map data: &copy; <a href="https://opentopomap.org>OpenTopoMap<\a> contributors')
m.add_layer(topo_layer)
m


Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

In [11]:
from ipyleaflet import ScaleControl

scale = ScaleControl(position='bottomleft')
m.add_control(scale)
m

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

In [20]:
geographic_data = {
    "City":["Varna", "Sofia"], 
    "Latitude":[43.2141, 42.6977], 
    "Longitude":[27.9147, 23.3219]
}

geo_df = gpd.GeoDataFrame(geographic_data, geometry=gpd.points_from_xy(geographic_data["Longitude"], geographic_data["Latitude"]))
geo_df

,City,Latitude,Longitude,geometry
0,Varna,43.2141,27.9147,POINT (27.9147 43.2141)
1,Sofia,42.6977,23.3219,POINT (23.3219 42.6977)


In [21]:
geo_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   City       2 non-null      object  
 1   Latitude   2 non-null      float64 
 2   Longitude  2 non-null      float64 
 3   geometry   2 non-null      geometry
dtypes: float64(2), geometry(1), object(1)
memory usage: 196.0+ bytes


In [22]:
coords_varna = (geo_df.loc[0,"Latitude"],geo_df.loc[0,"Longitude"])
coords_sofia = (geo_df.loc[1,"Latitude"],geo_df.loc[1,"Longitude"])
print(coords_varna)
print(coords_sofia)
distance = geodesic(coords_varna,coords_sofia).kilometers
print(f"Distance between Varna and Sofia: {distance} km")

(43.2141, 27.9147)
(42.6977, 23.3219)
Distance between Varna and Sofia: 379.08385751093476 km


In [35]:
# Find the midpoint between the cities' coordinates where the line will go through
# The midpoint is calculated by taking the mean of the latitude and longitude values from the DataFrame 'geo_df'
midpoint = [(geo_df["Latitude"].mean()), (geo_df["Longitude"].mean())]

# Create a Folium map object centered at the midpoint of the cities, with an initial zoom level of 5
vs_map = folium.Map(location=midpoint, zoom_start=5)

# Loop through each row in the 'geo_df' DataFrame to add markers to the map
for idx, row in geo_df.iterrows():
    folium.Marker(
        location=[row["Latitude"], row["Longitude"]],  # Set the marker location using latitude and longitude from 'geo_df'
        popup=row["City"]  # Display the city name as a popup on the marker
    ).add_to(vs_map)  # Add the marker to the Folium map

# Create a polyline (a line connecting points) between two specified cities using their latitude and longitude
line = folium.PolyLine(
    locations=[coords_varna, coords_sofia],  # Starting and Ending point of the line
    color="blue",  # Set the color of the line to blue
    weight=4,    # Set the thickness of the line
    opacity=0.5      # Set the opacity of the line to fully opaque
)

# Add the polyline to the Folium map
vs_map.add_child(line)